In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from keras.models import Sequential
from keras.layers import LSTM,Dense,Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.multiclass import OneVsRestClasssifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [2]:
!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip

--2023-07-02 06:34:53--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2023-07-02 06:34:53--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4931a72c475eec8b3d2490702c.dl.dropboxusercontent.com/cd/0/inline/B_Gft2d-_-yyxPIMeswMxxxf3InlgdyEJC6Yjr0PkoBgsTNZ_QW_8ZW_tIw0QjCAdwbj_CrM8YvZhSEG0sApT64TVaapLqiEpyeF8sqlu3AM4LsoRFZBApBLv637Z911_73AgMdfC_3PBg-c7rcaSZn5Pf6CkmogKG6eBZIRxlINZg/file# [following]
--2023-07-02 06:34:53--  https://uc4931a72c475eec8b3d2490702c.dl.dropboxusercontent.com/cd/0/inline/B_Gft2d-_-yyxPIMeswMxxxf3InlgdyEJC6Yjr0PkoBgsTNZ_QW_8ZW_tIw0QjCAdwbj_Cr

In [3]:
!unzip stacksample

Archive:  stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [4]:
data = pd.read_csv('Questions.csv', encoding='ISO-8859-1')
tags_df = pd.read_csv('Tags.csv', encoding='ISO-8859-1')
answers_df = pd.read_csv('Answers.csv', encoding='ISO-8859-1')

In [ ]:
# # Clean the text data
# def clean_text(text):
#    text = re.sub(r"<.*?>", "", text)
#    text = re.sub(r"\s+", " ", text)
#    return text

# data["Title"] = data["Title"].apply(clean_text)
# data["Body"] = data["Body"].apply(clean_text)

In [5]:
data_tags = data.merge(tags_df, how='inner',left_on='Id',right_on='Id')

In [6]:
top_tags = tags_df['Tag'].value_counts().nlargest(10).index.tolist()

In [7]:
top_tags

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [8]:
filtered_data = data_tags[data_tags['Tag'].isin(top_tags)]

In [9]:
#  Preprocess the text data
def preprocess_text(text):
    # Remove HTML tags
    text = re.sub('<[^>]+>', '', text)
    # Remove special characters and digits
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespaces
    text = re.sub('\s+', ' ', text).strip()
    return text


In [10]:
filtered_data['Title'] = filtered_data['Title'].apply(preprocess_text)
filtered_data['Body'] = filtered_data['Body'].apply(preprocess_text)

<ipython-input-10-0838d4b92558>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Title'] = filtered_data['Title'].apply(preprocess_text)
<ipython-input-10-0838d4b92558>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Body'] = filtered_data['Body'].apply(preprocess_text)


In [11]:
# Split the data into train and validation sets
X = filtered_data['Title'] + ' ' + filtered_data['Body']
y = filtered_data['Tag']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Vectorize the text data
# vectorizer = TfidfVectorizer(max_features=5000)
# X_train_vec = vectorizer.fit_transform(X_train)
# X_val_vec = vectorizer.transform(X_val)

In [12]:
# Transform tags into binary matrix representation
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train.apply(lambda x: [x]))
y_val_bin = mlb.transform(y_val.apply(lambda x: [x]))

In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train)

In [14]:
X_train_vec = tokenizer.texts_to_sequences(X_train)
X_val_vec = tokenizer.texts_to_sequences(X_val)

In [ ]:
# mlb= MultiLabelBinarizer()
# Y_train= mlb.fit_transform(X_train)['Tags'].apply(lambda x: x.split('|'))
# Y_val=mib.transform(X_val['Tags'].apply(lambda x: x.split('|')))

In [18]:
vocab_size=len(tokenizer.word_index) + 1
embedding_dim = 100

In [ ]:
input_length = 100
X_train_vec = pad_sequences(X_train_vec , padding='post')
X_val_vec = pad_sequences(X_val_vec, padding='post')

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequences_length))
model.add(LSTM(128))
model.add(Dense(len(top_tags), activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_vec, y_train_bin, epochs=10, batch_size=64)

In [ ]:
y_pred = model.predict(X_val_vec)
y_pred_labels = mlb.inverse_transform(y_pred.round())
y_val_labels = mlb.inverse_transform(y_val_bin )

In [ ]:
accuracy = accuracy_score(y_val_labels, y_pred_labels)
precision = precision_score(y_val_labels, y_pred_labels, average='micro')
recall = recall_score(y_val_labels, y_preds)